In [ ]:
%matplotlib inline
import numpy as np
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import skimage
import scipy
import os
import queue
import scipy.ndimage
import datetime
import random as rd
from skimage.transform import SimilarityTransform, warp
from skimage import transform
from pylab import *
from scipy import signal
from scipy import *
from PIL import Image
import keras 
from keras.datasets import mnist
from keras import models
from keras import layers
from keras.utils import to_categorical
from keras.models import load_model

In [ ]:
def ReadImage(path, discard=20):
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img[discard:-discard, discard:-discard, :], gray[discard:-discard, discard:-discard]

In [ ]:
def ShowImage(img, cmap=None):
    fig = plt.figure(dpi=150)
    a = fig.add_subplot(1, 1, 1)
    imgplot = plt.imshow(img, cmap=cmap)

In [ ]:
def GetContours(canny_edges):
    img, contours, hierarchy = cv2.findContours(canny_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = [(cv2.convexHull(cont), cv2.contourArea(cv2.convexHull(cont))) for cont in contours]
    contours.sort(key=lambda x: x[1], reverse=True)
    return [contours[0][0]]

In [ ]:
def ExtractSudokuCells(img, img_gray):
    k = np.zeros((11, 11))
    k[:, :] = 1.0 / 121
    m = scipy.ndimage.convolve(img_gray, k, mode='mirror')
    img_gray_blur = np.where(img_gray > m - 20, 255, 0).astype(np.uint8)
    img_gray_blur = cv2.GaussianBlur(img_gray_blur, (3, 3), 0)
    ShowImage(img_gray_blur, cmap='gray')
    edges = cv2.Canny(img_gray_blur, 50, 100)
    ShowImage(edges, cmap='gray')

    # generate contours
    contours = GetContours(edges)
    img_cont = cv2.drawContours(np.copy(img), contours, -1, (0, 255, 0), 3)
    ShowImage(img_cont)

    '''
    # draw contours on empty canvas
    pure_cont = cv2.drawContours(np.zeros(img.shape).astype(np.uint8), contours, -1, (0, 255, 0), 3)
    for p in contours[0]:
        x, y = p[0][1], p[0][0]
        pure_cont[x-2:x+3, y-2:y+3, 0] = np.ones((5,5)).astype(np.uint8) * 255
        pure_cont[x-2:x+3, y-2:y+3, 1] = np.zeros((5,5)).astype(np.uint8)

    x, y = contours[0][0][0][1], contours[0][0][0][0]
    pure_cont[x-2:x+3, y-2:y+3, 2] = np.ones((5,5)).astype(np.uint8) * 255
    pure_cont[x-2:x+3, y-2:y+3, 1] = np.zeros((5,5)).astype(np.uint8)
    pure_cont[x-2:x+3, y-2:y+3, 0] = np.zeros((5,5)).astype(np.uint8)
    ShowImage(pure_cont)
    '''

    # Get poly corners from contours
    for i in range(1, 1000):
        epsilon = i * 0.001 * cv2.arcLength(contours[0], True)
        approx = cv2.approxPolyDP(contours[0], epsilon, True)
        if len(approx) <= 4:
            approx = np.array([n[0] for n in approx]).astype(np.float32)
            break

    x_idx = np.argsort(approx[:, 0])
    first_2_x = approx[x_idx[:2], :]
    y_idx = np.argsort(first_2_x[:, 1])
    p00, p01 = x_idx[y_idx]
    last_2_x = approx[x_idx[2:], :]
    y_idx = np.argsort(last_2_x[:, 1]) + 2
    p10, p11 = x_idx[y_idx]

    approx = approx[[p00, p10, p11, p01], :]

    MARGIN_SIZE = 6

    # solve transformation
    h = np.array([[0, 0], [449, 0], [449, 449], [0, 449]]).astype(np.float32)
    transform = cv2.getPerspectiveTransform(approx, h)
    warp = cv2.cvtColor(cv2.warpPerspective(img, transform, (450 + MARGIN_SIZE, 450 + MARGIN_SIZE)), cv2.COLOR_BGR2GRAY)
    ShowImage(warp, cmap='gray')

    # Get cells
    cells = []
    for i in range(9):
        for j in range(9):
            x1, y1 = i * 50, (i + 1) * 50 + MARGIN_SIZE
            x2, y2 = j * 50, (j + 1) * 50 + MARGIN_SIZE
            cells.append(warp[x1:y1, x2:y2])

    return cells

In [ ]:
def DFS(cells, row, col, grid, row_idx=0):
    for i in range(row_idx, 9):
        for j in range(9):
            if cells[i][j] == 0:
                grid_x, grid_y = i // 3, j // 3
                next_row_idx = row_idx + 1 if j == 8 else row_idx
                for k in range(1, 10):
                    if not row[i][k] and not col[j][k] and not grid[grid_x][grid_y][k]:
                        row[i][k] = True
                        col[j][k] = True
                        grid[grid_x][grid_y][k] = True
                        cells[i][j] = k

                        if DFS(cells, row, col, grid, next_row_idx):
                            return True

                        row[i][k] = False
                        col[j][k] = False
                        grid[grid_x][grid_y][k] = False
                        cells[i][j] = 0
                return False
    return True

In [ ]:
def SolveSudoku(arr):
    row = np.zeros((10, 10), dtype=np.bool_)
    col = np.zeros((10, 10), dtype=np.bool_)
    grid = np.zeros((3, 3, 10), dtype=np.bool_)

    for i in range(9):
        for j in range(9):
            if arr[i][j] != 0:
                if row[i][arr[i][j]] or col[j][arr[i][j]] or grid[i // 3][j // 3][arr[i][j]]:
                    return False
                row[i][arr[i][j]] = True
                col[j][arr[i][j]] = True
                grid[i // 3][j // 3][arr[i][j]] = True

    return DFS(arr, row, col, grid)

In [ ]:
def ExtractConnectedComponent(img, size):
    MAN_DIST = 6
    dx = [0, 0, -1, 1, 1, 1, -1, -1]
    dy = [-1, 1, 0, 0, 1, -1, 1, -1]
    st_x, st_y = None, None

    explored_set = set()
    explored_set.add((size // 2, size // 2))
    que = queue.Queue()
    que.put((size // 2, size // 2))
    while not que.empty():
        pos_x, pos_y = que.get()
        if not abs(img[pos_x, pos_y]) < 1e-4: # not zero
            st_x, st_y = pos_x, pos_y
            break

        for i in range(4):
            x, y = pos_x + dx[i], pos_y + dy[i]
            if x >= 0 and x < size and y >= 0 and y < size and \
                    abs(x - size // 2) + abs(y - size // 2) < MAN_DIST and (x, y) not in explored_set:
                explored_set.add((x, y))
                que.put((x, y))

    if st_x is None:
        return None

    explored_set = set()
    explored_set.add((st_x, st_y))
    que = queue.Queue()
    que.put((st_x, st_y))

    avg_x, avg_y = 0.0, 0.0

    while not que.empty():
        pos_x, pos_y = que.get()
        avg_x += pos_x
        avg_y += pos_y

        for i in range(8):
            x, y = pos_x + dx[i], pos_y + dy[i]
            if x >= 0 and x < size and y >= 0 and y < size and abs(img[x, y] - 1) < 1e-4 and (x, y) not in explored_set:
                explored_set.add((x, y))
                que.put((x, y))

    if len(explored_set) < 10:
        return None

    avg_x = int(avg_x / len(explored_set))
    avg_y = int(avg_y / len(explored_set))
    pic = np.zeros((size, size))
    for (x, y) in explored_set:
        xx, yy = x + (size // 2 - avg_x), y + (size // 2 - avg_y)
        if xx >= 0 and xx < size and yy >= 0 and yy < size:
            pic[xx, yy] = 1.0
    return pic

In [ ]:
def RecognizeDigit(model_path, cells):
    CELL_SIZE = 28

    model = load_model(model_path)

    x = np.array(cells) / 255.0
    x = transform.resize(x, (len(cells), CELL_SIZE, CELL_SIZE, 1))

    for i in range(x.shape[0]):
        tmp = np.copy(x[i, :, :, 0])
        k = np.zeros((7, 7))
        k[:, :] = 1.0 / 49
        m = scipy.ndimage.convolve(tmp, k, mode='mirror')
        x[i, :, :, 0] = np.where(tmp > m - 0.05, 1.0, 0.0)

    for i in range(x.shape[0]):
        a, b = x.shape[1], x.shape[2]
        for j in range(3):
            x[i, j, :, 0] = 1.0
            x[i, a - j - 1, :, 0] = 1.0
            x[i, :, j, 0] = 1.0
            x[i, :, b - j - 1, 0] = 1.0
        #x[i, :, :, 0] = cv2.GaussianBlur(x[i, :, :, 0], (3, 3), 0)
    x = 1 - x

    for i in range(x.shape[0]):
        tmp = ExtractConnectedComponent(x[i, :, :, 0], CELL_SIZE)
        if tmp is not None:
            x[i, :, :, 0] = tmp
        else:
            x[i, :, :, 0] = np.zeros((CELL_SIZE, CELL_SIZE))

    fig = plt.figure(dpi=150)
    for i in range(9):
        for j in range(9):
            a = fig.add_subplot(9, 9, i * 9 + j + 1)
            plt.imshow(x[i * 9 + j, :, :, 0], cmap='gray')
    plt.show()
    
    # predicting
    results = model.predict(x)
    results = np.argmax(results, axis=1)

    for i in range(x.shape[0]):
        m = np.mean(x[i, 4:24, 4:24, 0])
        if m < 0.005:
            results[i] = 0
    return results

In [ ]:
if __name__ == '__main__':
    filepath = 'data/sudoku/sudoku3.png'
    filepath = 'data/sudoku/sudoku4.png'
    filepath = 'data/sudoku/sudoku1.jpg'
    filepath = 'data/sudoku/sudoku2.jpg'
    filepath = 'data/sudoku/sudoku6.jpg'
    filepath = 'data/sudoku/sudoku9.jpg'

    img, img_gray = ReadImage(filepath)

    cells = ExtractSudokuCells(img, img_gray)
    fig = plt.figure(dpi=150)
    for i in range(9):
        for j in range(9):
            a = fig.add_subplot(9, 9, i * 9 + j + 1)
            plt.imshow(cells[i * 9 + j], cmap='gray')
    plt.show()

    arr = RecognizeDigit('data/my_model.h5', cells)
    for i in range(9):
        for j in range(9):
            print(arr[i * 9 + j], end=' ')
        print()
        
    print()

    #### TEST Sudoku Solver ####
    arr_solution = [
        [0, 0, 0, 6, 0, 4, 7, 0, 0],
        [7, 0, 6, 0, 0, 0, 0, 0, 9],
        [0, 0, 0, 0, 0, 5, 0, 8, 0],
        [0, 7, 0, 0, 2, 0, 0, 9, 3],
        [8, 0, 0, 0, 0, 0, 0, 0, 5],
        [4, 3, 0, 0, 1, 0, 0, 7, 0],
        [0, 5, 0, 2, 0, 0, 0, 0, 0],
        [3, 0, 0, 0, 0, 0, 2, 0, 8],
        [0, 0, 2, 3, 0, 1, 0, 0, 0],
    ]
    ############################

    arr_ =[[arr[i*9+j] for j in range(9)] for i in range(9)]
    flag = SolveSudoku(arr_)
    if not flag:
        print('No solution')
    else:
        for i in range(9):
            for j in range(9):
                print(arr_[i][j], end=' ')
            print()

##### 